In [ ]:
from os import walk 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from scipy import signal 
from scipy import integrate
import re
import openpyxl
import pandas as pd
import math
import matplotlib.ticker as ticker
from scipy.optimize import curve_fit

os.chdir("C:\\Users\\Ilunya\\Desktop\\python\\wav zne") 
wdir='.'
for (dirpath, dirnames, filenames) in walk(wdir): 
    print()
def FIV3(mas): #calculation of FIV3
    ind=0
    #FIV3 max
    max1=0
    max2=0
    max3=0
    for i in range (0, len(mas)):
        if mas[i]>max1:
                max1=round(mas[i],5)
                idmax1=i
                ind=i+1
    flag=0
    for i in range (ind, len(mas)):
        if mas[i]<0: flag=1
        if mas[i]>max2 and flag==1:
            max2=round(mas[i],5)
            idmax2=i
            ind=i+1 
    flag=0
    for i in range (ind, len(mas)):
        if mas[i]<0: flag=1
        if mas[i]>max3 and flag==1:
            max3=round(mas[i],5) 
            idmax3=i
    #FIV3 min 
    min1=0
    min2=0
    min3=0
    for i in range (0, len(mas),1):
        if mas[i]<min1:
                min1=round(mas[i],5)
                idmin1=i
                ind=i+1
    flag=0
    for i in range (ind, len(mas),1):
        if mas[i]>0: flag=1
        if mas[i]<min2 and flag==1:
            min2=round(mas[i],5)
            idmin2=i
            ind=i+1 
    flag=0
    for i in range (ind, len(mas),1):
        if mas[i]>0: flag=1
        if mas[i]<min3 and flag==1:
            min3=round(mas[i],5)  
            idmin3=i
    FIV3max=round(max1+max2+max3,5)
    FIV3min=round(min1+min2+min3,5)
    fiv3=max(FIV3max,abs(FIV3min))
    return fiv3
def PGA(mas): #calculation of PGA ond other measures
    maxPGA=0
    for i in range(0,len(mas)):
        if abs(mas[i])>maxPGA: maxPGA=abs(mas[i])
    return maxPGA
flag=0
data_dict ={}
for name in filenames: #accelerogram files
    path=name 
    file = open(path) 
    strings = file.readlines() 
    firststring = strings[1].split() 
    thirdstring = strings[3].split() 
    file.close()
    if "acceleration" not in firststring: continue #checking for correctness of files
    flag+=1
    name=name[:-4]
    params=name.split('_') #taking info from filename
    M = float(params[-3]) #local magnitude
    H = float(params[-2]) #earthquake depth
    R = float(params[-1]) #epicentral distance
    Date = params[0] #earthquake date
    StName = params[1] #station code
    StType = params[2] #station type
    fiv3=[] #fiv3 massive
    PGA #peak ground acceleration
    PGV=0.0 #peak ground velocity
    Ia=0.0 #Arias intensity
    MFAS=[] #raw spectrum
    PMFAS=0.0 #peak raw spectrum
    freq=[] #frequensy from raw spectrum file
    f001=[] #peak raw spectrum (freq=0.01)
    f002=[]
    f003=[]
    f005=[]
    f0075=[]
    f01=[]
    f015=[]
    f02=[]
    f025=[]
    f03=[]
    f04=[]
    f05=[]
    f075=[]
    f1=[]
    f15=[]
    f2=[]
    f3=[]
    f4=[]
    f5=[]
    f75=[]
    f10=[]
    file = open(path)
    strings = file.readlines()
    thirdstring = strings[3].strip().split()
    file.close()
    count=round(float(thirdstring[0])/7-7) #count of data in one file
    step=float(thirdstring[1])
    data=np.loadtxt(path,skiprows=4,max_rows=count).flatten() #taking data from accelerograms
    timesMin=np.array([(step*i)/60 for i in range(len(data))]) # taking time and converting from nm to sec
    times=np.array([step*i for i in range(len(data))]) 

    data_dict[name] = [times,data,str(M),str(H),str(R), fiv3, step, Date, StName, StType, PGA, PGV, Ia, RS, PRS, 
                        #0    1     2       3      4      5      6     7     8        9     10   11  12  13   14
                       f001, f002, f003, f005, f0075, f01, f015, f02,
                       #15    16    17     18     19   20    21   22    
                       f025, f03, f04, f05, f075, f1, f15, f2, f3, f4, f5, f75, f10,
                      #23     24   25   26   27   28   29   30  31 32  33   34   35
                       MFAS, PMFAS,freq] #dictionary with all data from earthquake
                        #36   37   38    
os.chdir("C:\\Users\\Ilunya\\Desktop\\specs") #taking data from raw spectrum files
wdir='.'        
for (dirpath, dirnames, filenames) in walk(wdir): 
    print()
for name in filenames: 
    path=name 
    file = open(path) 
    strings = file.readlines() 
    specs=[] 
    s=1
    for x in strings: 
        x=re.sub(" +", " ", x)
        if s==1:
            s=2
            continue
        specs.append(float(x.split(' ')[2]))
    data_dict[name.replace(".out", "")][36]=specs #add spectum to dictionary
    data_dict[name.replace(".out", "")][38]=freq #add frequensy to dictionary
    file.close()
for k in data_dict.keys(): 
    sig = data_dict[k][1]
    for i in range(0, len(sig)):
        sig[i]*=10**(-7) #converting from nm/s to cm/s
    t=data_dict[k][0]
    sos = signal.butter(2, 1, 'low', fs=50, output='sos') #2nd order lowpass Butterworth filter with freq=1
    filtered = signal.sosfilt(sos, sig)
    data_dict[k][10]=round(PGA(data_dict[k][1]),5) #add FIV3 massive to dictionary
    integrated=[]
    integrated = integrate.cumtrapz(data_dict[k][1], data_dict[k][0], initial=0)
    data_dict[k][11]=round(PGA(integrated),5)
    FAS=[]
    for l in range(0,len(data_dict[k][36])):
        FAS.append((10**data_dict[k][36][l])*10**(-9)) #converting from nm/s to m/s
    MFAS.append(round(PGA(FAS),3))
    data_dict[k][37]=MFAS #add peak raw spectrum to dictionary
    integrated=[] #null mass
    Ia=[]
    mas=[]
    PIa=0
    g=9.81
    deltat=data_dict[k][0][1]+0.2
    limit=50
    integrated = integrate.cumtrapz((data_dict[k][1]*10**(-2))**2, data_dict[k][0], initial=0) #converting from cm/s to m/s and integrate
    for i in range(0,len(integrated)):
        mas.append(integrated[i]*np.pi/(2*g)) #use Arias intensity formula
    Ia=mas
    data_dict[k][12]=PGA(Ia) #add Arias intensity to dictionary
    data_dict[k][14]=round(PGA(data_dict[k][13]),5) #add PMFAS to dictionary
    T=0.01
    integrated=[]
    deltat=data_dict[k][0][1]+0.7*T
    limit=30 #creating limit for integrating without mistakes
    for i in range(0,len(filtered)-limit):
        t=data_dict[k][0]
        c=0
        j=0
        sumt=0
        while sumt<deltat: #integrating by sum
            c+=filtered[i+j]*(t[i+j+1]-t[i+j])
            sumt+=t[i+j+1]-t[i+j]
            j+=1
        integrated.append(c)
    data_dict[k][5].append(FIV3(integrated[:-limit])) #add FIV3 T=0.01 to dictionary
    T=0.2
    integrated=[]
    deltat=data_dict[k][0][1]+0.7*T
    limit=40 #creating limit bigger because step for sum became higher
    for i in range(0,len(filtered)-limit):
        t=data_dict[k][0]
        c=0
        j=0
        sumt=0
        while sumt<deltat:
            c+=filtered[i+j]*(t[i+j+1]-t[i+j])
            j+=1
            sumt+=t[i+j+1]-t[i+j]
        integrated.append(c)
    data_dict[k][5].append(FIV3(integrated[:-limit])) #add FIV3 T=0.2 to dictionary
    T=1
    integrated=[]
    deltat=data_dict[k][0][1]+0.7*T
    limit=150
    for i in range(0,len(filtered)-limit):
        t=data_dict[k][0]
        c=0
        j=0
        sumt=0
        while sumt<deltat:
            c+=filtered[i+j]*(t[i+j+1]-t[i+j])
            j+=1
            sumt+=t[i+j+1]-t[i+j]
        integrated.append(c)
    data_dict[k][5].append(FIV3(integrated[:-limit])) #add FIV3 T=1 to dictionary
    T=3
    integrated=[]
    deltat=data_dict[k][0][1]+0.7*T
    limit=550
    for i in range(0,len(filtered)-limit):
        t=data_dict[k][0]
        c=0
        j=0
        sumt=0
        while sumt<deltat:
            c+=filtered[i+j]*(t[i+j+1]-t[i+j])
            j+=1
            sumt+=t[i+j+1]-t[i+j]
        integrated.append(c)
    data_dict[k][5].append(FIV3(integrated[:-limit])) #add FIV3 T=3 to dictionary
    fr=data_dict[k][38] 
    for j in range(0,len(fr)): 
        if round(fr[j],2)==0.01: data_dict[k][15]=round(FAS[j],6) #add Sa freq=0.01 to dictionary
        if round(fr[j],2)==0.02: data_dict[k][16]=round(FAS[j],6) #freq=0.02
        if round(fr[j],2)==0.03: data_dict[k][17]=round(FAS[j],6) #freq=0.03
        if round(fr[j],2)==0.05: data_dict[k][18]=round(FAS[j],6) #freq=0.05
        if round(fr[j],3)>=0.071 and round(fr[j],3)<=0.079: data_dict[k][19]=round(FAS[j],6) #freq=0.075
        if round(fr[j],1)==0.1: data_dict[k][20]=round(FAS[j],6) #freq=0.1
        if round(fr[j],2)>=0.14 and round(fr[j],2)<=0.16: data_dict[k][21]=round(FAS[j],6) #freq=0.15
        if round(fr[j],1)==0.2: data_dict[k][22]=round(FAS[j],6) #freq=0.2
        if round(fr[j],2)>=0.24 and round(fr[j],2)<=0.26: data_dict[k][23]=round(FAS[j],6) #freq=0.25
        if round(fr[j],1)==0.3: data_dict[k][24]=round(FAS[j],6) #freq=0.3
        if round(fr[j],1)==0.4: data_dict[k][25]=round(FAS[j],6) #freq=0.4
        if round(fr[j],1)==0.5: data_dict[k][26]=round(FAS[j],6) #freq=0.5
        if round(fr[j],2)>=0.73 and round(fr[j],2)<=0.77: data_dict[k][27]=round(FAS[j],6) #freq=0.75
        if round(fr[j],1)==1: data_dict[k][28]=round(FAS[j],6) #freq=1
        if round(fr[j],1)>=1.4 and round(fr[j],1)<=1.6: data_dict[k][29]=round(FAS[j],6) #freq=1.5
        if round(fr[j],1)==2: data_dict[k][30]=round(FAS[j],6) #freq=2
        if round(fr[j],1)==3: data_dict[k][31]=round(FAS[j],6) #freq=3
        if round(fr[j],1)==4: data_dict[k][32]=round(FAS[j],6) #freq=4
        if round(fr[j],1)==5: data_dict[k][33]=round(FAS[j],6) #freq=5
        if round(fr[j],1)>=7.4 and round(fr[j],1)<=7.6: data_dict[k][34]=round(FAS[j],6) #freq=7.5
        if round(fr[j],1)==10: data_dict[k][35]=round(FAS[j],6) #freq=10
os.chdir("C:\\Users\\Ilunya\\Desktop\\Python")

for i in range(0,len(rData)):
    rhyp=round(np.sqrt(rData[i]**2+hData[i]**2),2)
    rhData.append(rhyp)
    mwData.append(0.05 * mData[i]**3 - 0.64 * mData[i]**2 + 3.5 * mData[i]-2.89)

for k in (data_dict.keys()):
    rData=np.append(rData,float(data_dict[k][4]))
    mData=np.append(mData,float(data_dict[k][2]))
    hData=np.append(hData,float(data_dict[k][3]))
    f1Data=np.append(f1Data,data_dict[k][5][0])
    f2Data=np.append(f2Data,data_dict[k][5][1])
    f3Data=np.append(f3Data,data_dict[k][5][2])
    f4Data=np.append(f4Data,data_dict[k][5][3])
    aData=np.append(aData,data_dict[k][10])
    vData=np.append(vData,data_dict[k][11])
    iData=np.append(iData,data_dict[k][12])
    sData=np.append(sData,data_dict[k][14]) 
    dData=np.append(dData,data_dict1[k][0])
    stData=np.append(stData,data_dict[k][8])
    sttData=np.append(sttData,data_dict[k][9])
    tData.append(data_dict1[k][1])
    f001.append(data_dict[k][15])
    f002.append(data_dict[k][16])
    f003.append(data_dict[k][17])
    f005.append(data_dict[k][18])
    f0075.append(data_dict[k][19])
    f01.append(data_dict[k][20])
    f015.append(data_dict[k][21])
    f02.append(data_dict[k][22])
    f025.append(data_dict[k][23])
    f03.append(data_dict[k][24])
    f04.append(data_dict[k][25])
    f05.append(data_dict[k][26])
    f075.append(data_dict[k][27])
    f1.append(data_dict[k][28])
    f15.append(data_dict[k][29])
    f2.append(data_dict[k][30])
    f3.append(data_dict[k][31])
    f4.append(data_dict[k][32])
    f5.append(data_dict[k][33])
    f75.append(data_dict[k][34])
    f10.append(data_dict[k][35])

#saving all data in one npz file:
np.savez('all data', mData=mData, rData=rData, hData=hData, f1Data=f1Data,f2Data=f2Data,f3Data=f3Data,f4Data=f4Data,
         aData=aData,vData=vData, iData=iData, sData=sData,dData=dData,stData=stData,sttData=sttData,mwData=mwData,
         rhData=rhData,fName=fName,tData=tData, f001=f001,f002=f002,f003=f003,f005=f005,f0075=f0075,
         f01=f01,f015=f015,f02=f02,f03=f03,f04=f04,f05=f05,f075=f075,f1=f1,f15=f15,f2=f2,f3=f3,f4=f4,f5=f5,f75=f75,
         f10=f10, MFAS=MFAS ) 

#choosing max measure from horizontal components:
while i<len(mwData-1): 
    j=i
    while j<len(mwData-1):
        if j==285: break
        if dData[i]==dData[j] and tData[i]==tData[j] and rData[i]==rData[j] and sttData[i]!=sttData[j] and sttData[i]!='SHZ' and sttData[i]!='EHZ' and sttData[j]!='SHZ' and sttData[j]!='EHZ' and sttData[i]!='HZ' and sttData[j]!='HZ':
            #print(sttData[i])
            #print(i)
            if aData[i]>aData[j]: ij=i
            else: ij=j
            d1Data.append(dData[ij])
            f11Data.append(f1Data[ij])
            f21Data.append(f2Data[ij])
            f31Data.append(f3Data[ij])
            f41Data.append(f4Data[ij])
            a1Data.append(aData[ij])
            v1Data.append(vData[ij])
            i1Data.append(iData[ij])
            s1Data.append(np.log10(np.sqrt((10**sData[i])**2+(10**sData[j])**2)))
            m1Data.append(mData[ij])
            mw1Data.append(mwData[ij])
            r1Data.append(rData[ij])
            rh1Data.append(rhData[ij])
            stt1Data.append(sttData[ij])
            f001x.append(f001[ij])
            f002x.append(f002[ij])
            f003x.append(f003[ij])
            f005x.append(f005[ij])
            f0075x.append(f0075[ij])
            f01x.append(f01[ij])
            f015x.append(f015[ij])
            f02x.append(f02[ij])
            f025x.append(f025[ij])
            f03x.append(f03[ij])
            f04x.append(f04[ij])
            f05x.append(f05[ij])
            f075x.append(f075[ij])
            f1x.append(f1[ij])
            f15x.append(f15[ij])
            f2x.append(f2[ij])
            f3x.append(f3[ij])
            f4x.append(f4[ij])
            f5x.append(f5[ij])
            f75x.append(f75[ij])
            f10x.append(f10[ij])
            i+=1  

lon=[]
lat=[]
ID=[]
Mw=[]
for i in range (0, len(dData)):
    lon.append(0)
    lat.append(0)
    ID.append('')
    Mw.append(0)
    
#add earthquake info:
for i in range(0,len(mData)):
    if dData[i]=='22.08.2009' and mData[i]==5:
        ID[i]='kEDYmxgB'
        lat[i]='52,756'
        lon[i]='144,042'
        Mw[i]=4.8
    if dData[i]=='16.03.2010' and mData[i]==5.7:
        ID[i]='2OnYx7gy'
        lat[i]='52,081'
        lon[i]='142,175'
        Mw[i]=5.8
    if dData[i]=='09.07.2010' and mData[i]==5.1:
        ID[i]='9EBmd1Mz'
        lat[i]='52,114'
        lon[i]='142,154'
        Mw[i]=5
    if dData[i]=='06.03.2011' and mData[i]==4.6:
        ID[i]='QgpPLGgW'
        lat[i]='52,102'
        lon[i]='142,207'
        Mw[i]=4.9
    if dData[i]=='12.12.2011' and mData[i]==5.4:
        ID[i]='yMKwJqMW'
        lat[i]='50,688'
        lon[i]='142,969'
        Mw[i]=5.1
    if dData[i]=='21.10.2012' and mData[i]==4.8:
        ID[i]='xEw9o2Oy'
        lat[i]='53,405'
        lon[i]='142,55'
        Mw[i]=4.9
    if dData[i]=='19.02.2014' and mData[i]==5:
        ID[i]='kEedbGMY'
        lat[i]='52,15'
        lon[i]='143,361'
        Mw[i]=5.1
    if dData[i]=='30.06.2014' and mData[i]==4.5:
        ID[i]='WgvxrBMj'
        lat[i]='51,57'
        lon[i]='143,101'
        Mw[i]=4.9
    if dData[i]=='14.08.2016' and mData[i]==6.1:
        ID[i]='QgpAn7OW'
        lat[i]='50,351'
        lon[i]='142,395'
        Mw[i]=5.7
    if dData[i]=='14.08.2016' and mData[i]==4.5:
        ID[i]='lgbejpE7'
        lat[i]='50,31'
        lon[i]='142,344'
        Mw[i]=4.6
    if dData[i]=='15.08.2016' and mData[i]==5:
        ID[i]='zErvpXEG'
        lat[i]='50,376'
        lon[i]='142,388'
        Mw[i]=4.7
    if dData[i]=='16.08.2016' and mData[i]==4.6:
        ID[i]='vO4zDXEP'
        lat[i]='50,355'
        lon[i]='142,499'
        Mw[i]=4.6
    if dData[i]=='17.08.2016' and mData[i]==4.9:
        ID[i]='KEXK05M0'
        lat[i]='50,322'
        lon[i]='142,435'
        Mw[i]=4.6
    if dData[i]=='17.08.2016' and mData[i]==4.5:
        ID[i]='zO9L5egK'
        lat[i]='50,376'
        lon[i]='142,446'
        Mw[i]=4.6
    if dData[i]=='20.08.2016' and mData[i]==4.4:
        ID[i]='kEePppMY'
        lat[i]='50,213'
        lon[i]='142,816'
        Mw[i]=4.5
    if dData[i]=='07.04.2017' and mData[i]==4.1:
        ID[i]='xEwjZBMy'
        lat[i]='46,93'
        lon[i]='142,607'
        Mw[i]=''
    if dData[i]=='23.04.2017' and mData[i]==5.1:
        ID[i]='KEXKBvM3'
        lat[i]='46,178'
        lon[i]='141,976'
        Mw[i]=5.2
    if dData[i]=='08.06.2017' and mData[i]==4.3:
        ID[i]='WgvnwNgj'
        lat[i]='46,948'
        lon[i]='142,572'
        Mw[i]=4.4
    if dData[i]=='07.10.2018' and mData[i]==4.1:
        ID[i]='LbdmJPAE'
        lat[i]='51,324'
        lon[i]='142,167'
        Mw[i]=4.7
    if dData[i]=='23.01.2020' and mData[i]==4.4:
        ID[i]='xAm2x7B8'
        lat[i]='46,563'
        lon[i]='142,456'
        Mw[i]=4.4
    if dData[i]=='13.09.2020' and mData[i]==5:
        ID[i]='EAWDxgBk'
        lat[i]='48,963'
        lon[i]='142,032'
        Mw[i]=5
    if dData[i]=='08.11.2020' and mData[i]==4.7:
        ID[i]='oBOjzMAN'
        lat[i]='52,992'
        lon[i]='142,427'
        Mw[i]=4.6
    if dData[i]=='28.01.2021' and mData[i]==4.2:
        ID[i]='qB24vXbY'
        lat[i]='51,854'
        lon[i]='143,033'
        Mw[i]=4.1
    if dData[i]=='18.04.2021' and mData[i]==4.5:
        ID[i]='NBxeolZJ'
        lat[i]='50,87'
        lon[i]='142,621'
        Mw[i]=4.2
    if dData[i]=='17.06.2021' and mData[i]==4.3:
        ID[i]='Vb7WQObM'
        lat[i]='47,237'
        lon[i]='142,649'
        Mw[i]=''
    if dData[i]=='28.12.2021' and mData[i]==4.1:
        ID[i]='yBYzVYZn'
        lat[i]='50,523'
        lon[i]='143,921'
        Mw[i]=4.5
    if dData[i]=='05.02.2022' and mData[i]==5.5:
        ID[i]='dBGGg5By'
        lat[i]='52,527'
        lon[i]='143,452'
        Mw[i]=5.2
    if dData[i]=='09.02.2022' and mData[i]==4.1:
        ID[i]='6AqJQ8Br'
        lat[i]='46,82'
        lon[i]='142,115'
        Mw[i]=''

#saving all intensity measures to one excel file:
df = pd.DataFrame()
df = df.append(pd.DataFrame({
                            'ID': ID,
                            'Date': dData,
                            'Time': tData,
                            'Latitude': lat,
                            'Longitude': lon,
                            'Depth, km': hData,
                            'R_epi, km': rData,
                            'R_hyp, km': rhData,
                            'ML': mData,
                            'Mw (USGS)': Mw,
                            'Mw (calculation)': mwData,
                            'PGA, cm/s²': aData,
                            'PGV, cm/s': vData,
                            'Ia, m/s': iData,
                            'MFAS, m/s': MFAS,
                            'FAS (0.01), m/s': f001,
                            'FAS (0.02), m/s': f002,
                            'FAS (0.03), m/s': f003,
                            'FAS (0.05), m/s': f005,
                            'FAS (0.075), m/s': f0075,
                            'FAS (0.1), m/s': f01,
                            'FAS (0.2), m/s': f02,
                            'FAS (0.25), m/s': f025,
                            'FAS (0.3), m/s': f03,
                            'FAS (0.4), m/s': f04,
                            'FAS (0.5), m/s': f05,
                            'FAS (0.75), m/s': f075,
                            'FAS (1), m/s': f1,
                            'FAS (2), m/s': f2,
                            'FAS (3), m/s': f3,
                            'FAS (4), m/s': f4,
                            'FAS (5), m/s': f5,
                            'FAS (7.5), m/s': f75,
                            'FAS (10), m/s': f10,
                            'FIV3 (0.01)': f1Data,
                            'FIV3 (0.2)': f2Data,
                            'FIV3 (1)': f3Data,
                            'FIV3 (3)': f4Data,
                            'Код станции': stData,
                            'Канал': sttData,
                            'Имя файла': fName
                             }), ignore_index=True)
df.to_excel('./all data zne.xlsx')
len(mData)

#calculate gmpe coefficients :
allData=[]
allData.append(f1Data)
allData.append(f2Data)
allData.append(f3Data)
allData.append(f4Data)
allData.append(vData)
allData.append(aData)
allData.append(iData)
allData.append(sData)
def log10ValRh(m1): #function for logarithm massives
    res=[]
    for i in range(0,len(m1)):
        res.append(np.log10(m1[i]))
    return res
def func(X, a, b, c, d):
    x1,x2=X
    return np.multiply(a,x1) - np.multiply(b,np.log10(x2)) - np.multiply(c,x2) + d
def func1(X, a, b, c, d):
    x1,x2=X
    return np.multiply(a,x1) - np.multiply(b,np.log10(x2)) - np.multiply(c,x2) + d
logs=[]
RMSE=[]
coeffs=[]
perr=[]
logs.append(log10ValRh(f11Data)) #FIV3 T=0.01
logs.append(log10ValRh(f21Data)) #FIV3 T=0.2
logs.append(log10ValRh(f31Data)) #FIV3 T=1
logs.append(log10ValRh(f41Data)) #FIV3 T=3
logs.append(log10ValRh(v1Data)) #Velocity
logs.append(log10ValRh(a1Data)) #Acceleration
logs.append(log10ValRh(i1Data)) #Arias Intentity
logs.append(s1Data) #Raw Spectrum
for i in range (0,len(logs)):
    yData=logs[i]
    xData=mw1Data,rh1Data
    if i==6:
        popt, pcov = curve_fit(func1, xData, yData)
    else:
        popt, pcov = curve_fit(func, xData, yData)
    coeffs.append(popt)
    print(coeffs[i])
    perr.append(np.sqrt(np.diag(pcov)))
    print(perr[i])
    modelPredictions = func(xData, *popt) 
    absError = modelPredictions - yData
    SE = np.square(absError) # squared errors
    MSE = np.mean(SE) # mean squared errors
    RMSE.append(np.sqrt(MSE))
    print(RMSE[i])
    from sklearn.metrics import r2_score
    r2.append(round(r2_score(yData, modelPredictions),3)) #calculate R² metrica
    print(r2[i])
a=[]
k=[]
b=[]
c=[]
rmse=[]
unit=["lg FIV3(Mw), T=0.01 [cm/s]","lg FIV3(Mw), T=0.2 [cm/s]","lg FIV3(Mw), T=1 [cm/s]","lg FIV3(Mw), T=3 [cm/s]","lg PGV(Mw) [cm/s]","lg PGA(Mw) [cm/s²]","lg Iₐ(Mw) [m/s]","lg MFAS(Mw) [m/s]"]
for i in range (0,len(coeffs)):
    a.append(str(round(coeffs[i][0],2))+"±"+str(round(perr[i][0],2)))
    k.append(str(round(coeffs[i][1],2))+"±"+str(round(perr[i][1],2)))
    b.append(str(round(coeffs[i][2],4))+"±"+str(round(perr[i][2],4)))
    if i==7: c.append(str(round(coeffs[i][3]-9,2))+"±"+str(round(perr[i][3],2)))
    else: c.append(str(round(coeffs[i][3],2))+"±"+str(round(perr[i][3],2)))
    rmse.append(str(round(RMSE[i],3)))
df = pd.DataFrame()
#saving coefficients to excel file:
df = df.append(pd.DataFrame({ 'unit': unit,
                            'a': a,
                            'k': k,
                            'b': b,  
                            'c': c, 
                            'σ': rmse
                            
                             }), ignore_index=True)
                    
           
df.to_excel('./coeffs abcd(mw).xlsx')